#### New to Plotly?
Plotly's Python library is free and open source! [Get started](https://plot.ly/python/getting-started/) by downloading the client and [reading the primer](https://plot.ly/python/getting-started/).
<br>You can set up Plotly to work in [online](https://plot.ly/python/getting-started/#initialization-for-online-plotting) or [offline](https://plot.ly/python/getting-started/#initialization-for-offline-plotting) mode, or in [jupyter notebooks](https://plot.ly/python/getting-started/#start-plotting-online).
<br>We also have a quick-reference [cheatsheet](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) (new!) to help you get started!

#### Network reflecting coappearances of characters in <br>Victor Hugo's novel Les Miserables

We define our graph as an `igraph.Graph` object. [Python `igraph`](http://igraph.org/python/)
is a library for high-performance graph generation  and analysis. Install the Python library with `sudo pip install python-igraph`.

In [1]:
import igraph as ig

ModuleNotFoundError: No module named 'igraph'

Read graph data from a `json` file:

In [3]:
#import json
#import urllib2
import requests

# data = []
# req = urllib2.Request("https://raw.githubusercontent.com/plotly/datasets/master/miserables.json")
# opener = urllib2.build_opener()
# f = opener.open(req)
#data = json.loads(f.read())

r = requests.get("https://raw.githubusercontent.com/plotly/datasets/master/miserables.json")
print(r.status_code)
data = r.json()
print(data.keys())

200
dict_keys(['nodes', 'links'])


Get the number of nodes:

In [4]:
N=len(data['nodes'])
N

77

Define the list of edges and the Graph object from Edges:

In [5]:
L=len(data['links'])
Edges=[(data['links'][k]['source'], data['links'][k]['target']) for k in range(L)]

print(dir(ig))
G=ig.Graph(Edges, directed=False)

['ADJ_DIRECTED', 'ADJ_LOWER', 'ADJ_MAX', 'ADJ_MIN', 'ADJ_PLUS', 'ADJ_UNDIRECTED', 'ADJ_UPPER', 'ALL', 'ARPACKOptions', 'AdvancedGradientPalette', 'BFSIter', 'BLISS_F', 'BLISS_FL', 'BLISS_FLM', 'BLISS_FM', 'BLISS_FS', 'BLISS_FSM', 'BoundingBox', 'ClusterColoringPalette', 'Clustering', 'CohesiveBlocks', 'Configuration', 'Cover', 'Cut', 'DefaultGraphDrawer', 'Dendrogram', 'DyadCensus', 'Edge', 'EdgeSeq', 'FittedPowerLaw', 'Flow', 'GET_ADJACENCY_BOTH', 'GET_ADJACENCY_LOWER', 'GET_ADJACENCY_UPPER', 'GradientPalette', 'Graph', 'GraphBase', 'GraphSummary', 'Histogram', 'IN', 'InternalError', 'Layout', 'Matching', 'Matrix', 'Nexus', 'NexusConnection', 'OUT', 'Palette', 'Plot', 'Point', 'PrecalculatedPalette', 'REWIRING_SIMPLE', 'REWIRING_SIMPLE_LOOPS', 'RainbowPalette', 'Rectangle', 'RunningMean', 'STAR_IN', 'STAR_MUTUAL', 'STAR_OUT', 'STAR_UNDIRECTED', 'STRONG', 'StringIO', 'TRANSITIVITY_NAN', 'TRANSITIVITY_ZERO', 'TREE_IN', 'TREE_OUT', 'TREE_UNDIRECTED', 'TriadCensus', 'UniqueIdGenerator', '

Extract the node attributes, 'group', and 'name':

In [6]:
data['nodes'][0]

{'name': 'Myriel', 'group': 1}

In [7]:
labels=[]
group=[]
for node in data['nodes']:
    labels.append(node['name'])
    group.append(node['group'])

Get the node positions, set by the Kamada-Kawai layout for 3D graphs:

In [8]:
layt=G.layout('kk', dim=3) 

`layt` is a list of three elements lists (the coordinates of nodes):

In [9]:
layt[5]

[-2.878183799245599, -7.953161721203702, 0.9682673355936177]

Set data for the Plotly plot of the graph:

In [10]:
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]  
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]  

In [11]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             color=group,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Network of coappearances of characters in Victor Hugo's novel<br> Les Miserables (3D visualization)",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text="Data source: <a href='http://bost.ocks.org/mike/miserables/miserables.json'>[1] miserables.json</a>",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ],    )

In [14]:
data=[trace1, trace2]
#fig=go.Figure(data=data, layout=layout)

fig = go.FigureWidget(data=data, layout=layout)
fig
#py.iplot(fig, filename='Les-Miserables')

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(125,125,125)', 'width':…

#### Reference
See https://plot.ly/python/reference/#scatter3d for more information and chart attribute options!

In [16]:
from IPython.display import display, HTML

display(HTML('<link href="//fonts.googleapis.com/css?family=Open+Sans:600,400,300,200|Inconsolata|Ubuntu+Mono:400,700" rel="stylesheet" type="text/css" />'))
display(HTML('<link rel="stylesheet" type="text/css" href="http://help.plot.ly/documentation/all_static/css/ipython-notebook-custom.css">'))

! pip install git+https://github.com/plotly/publisher.git --upgrade

import publisher
publisher.publish(
    'Les-miserables-network.ipynb', 'python/3d-network-graph/', 'Python 3D Network Graphs',
    'How to make 3D Network Graphs in Python. ',
    title = '3D Network Graphs in Python | plotly',
    name = '3D Network Graphs',
    has_thumbnail='true', thumbnail='thumbnail/3dnetwork.jpg', 
    language='python', page_type='example_index', 
    display_as='3d_charts', order=13,
    ipynb= '~notebook_demo/226')

  Cloning https://github.com/plotly/publisher.git to /tmp/pip-req-build-5nojxpvr
  Stored in directory: /tmp/pip-ephem-wheel-cache-we0glnoh/wheels/99/3e/a0/fbd22ba24cca72bdbaba53dbc23c1768755fb17b3af0f33966
Successfully built publisher
  Found existing installation: publisher 0.13
    Uninstalling publisher-0.13:
      Successfully uninstalled publisher-0.13
